In [4]:
import tkinter as tk
from tkinter import filedialog
from PIL import ImageTk, Image
import cv2
import dlib
import numpy as np

# PyTorch model and training necessities
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#Image datasets and image manipulation
import torchvision
from torchvision import datasets, models, transforms

from torch.nn import functional as F
from torchvision.models import resnet50, ResNet50_Weights
import glob
import warnings
from imutils import face_utils
from scipy.optimize import curve_fit
import os
warnings.filterwarnings('ignore')

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")


########
#DATASETS
#Pregătim dataseturile pentru a le putea utiliza

URL1 = "E:\\Profil\\Desktop\\Licenta\\ISIA\\LICENTA\\poze_eu_masca"
URL2 = "E:\\Profil\\Desktop\\Licenta\\ISIA\\LICENTA\\poze_eu_fara_masca"

def import_and_do_datasets(URL1, URL2):
    transform_test = transforms.Compose(
    [transforms.Resize((224,224)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])])


    #Importam dataset pentru demascare faciala, respectiv identificare parte superioara
    image_Datasets_WithMask = datasets.ImageFolder(URL1, transform = transform_test)
    image_Datasets_WithoutMask =  datasets.ImageFolder(URL2, transform = transform_test)

    #Cream data loader pentru datasetul cu demascarea 
    dataloaders_WithMask = torch.utils.data.DataLoader(image_Datasets_WithMask, batch_size = 1, shuffle = False, num_workers = 4)
    dataloaders_WithoutMask = torch.utils.data.DataLoader(image_Datasets_WithoutMask, batch_size = 1, shuffle = False, num_workers = 4)
    
    return transform_test, image_Datasets_WithMask, image_Datasets_WithoutMask, dataloaders_WithMask, dataloaders_WithoutMask
    

def model_mask_output(image_path, model_path):
    image = Image.open(image_path).convert('RGB')
    
    #model = models.resnet50(weights=ResNet50_Weights.DEFAULT).to(device) # cele mai actuale weights, dar are probleme 
    model = models.resnet50(pretrained=True).to(device)   #importam resnet


    # freezing pretrained layers
    for param in model.parameters():
        param.requires_grad = True 

    model.fc = nn.Sequential(
            nn.Dropout(p = 0.5, inplace = False),
            nn.Linear(in_features = 2048, out_features =128, bias = True),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5, inplace = False),
            nn.Linear(in_features = 128, out_features = 2, bias = True)).to(device)

    
    #Load the model
    model.load_state_dict(torch.load(model_path))
    model.eval()
    
    validation_image = torch.stack([transform_test(image).to(device)])
    
    pred_logits_tensor = model(validation_image)
    pred_probs = F.softmax(pred_logits_tensor, dim=1).cpu().data.numpy()
    
    if pred_probs[0, 0] >= 0.5:
        out = 1
    else:
        out = 0
    
    return out

def model_people(image_path, model_path):
    image = Image.open(image_path).convert('RGB')
    
    #model = models.resnet50(weights=ResNet50_Weights.DEFAULT).to(device) # cele mai actuale weights, dar are probleme 
    model_people = models.resnet50(pretrained=True).to(device)            #importam resnet


    # freezing pretrained layers
    for param in model_people.parameters():
        param.requires_grad = True 

    model_people.fc = nn.Sequential(
                nn.Dropout(p = 0.5, inplace = False),
                nn.Linear(in_features = 2048, out_features =128, bias = True),
                nn.ReLU(inplace=True),
                nn.Dropout(p = 0.5, inplace = False),
                nn.Linear(in_features = 128, out_features = 10, bias = True)).to(device)
    
    model_people.load_state_dict(torch.load(model_path))

    # Set the model to evaluation mode
    model_people.eval()
    
    validation_image = torch.stack([transform_test(image).to(device)])
    
    pred_logits_tensor = model_people(validation_image)
    pred_probs = F.softmax(pred_logits_tensor).cpu().data.numpy()
    
    return pred_probs

def check_person(vector):
    
    vector = vector.flatten()
    max_index = np.argmax(vector)
    
    return max_index

def replace_red_with_color(image):
    for y in range(w1):
        for x in range(h1):
            blue_channel  = image[y,x,0]
            green_channel = image[y,x,1]
            red_channel   = image[y,x,2]
            if red_channel >= 200 and green_channel >= 10 and green_channel <= 50 and blue_channel >= 55 :
                image[y,x] = [0,0,0]


def replace_cyan_with_color(image):
    for y in range(w1):
        for x in range(h1):
            blue_channel  = image[y,x,0]
            green_channel = image[y,x,1]
            red_channel   = image[y,x,2]
            if red_channel <= 75 and green_channel >= 100 and blue_channel >= 100 :
                image[y,x] = [0,0,0]   


def replace_green_with_color(image):
    for y in range(w1):
        for x in range(h1):
            blue_channel  = image[y,x,0]
            green_channel = image[y,x,1]
            red_channel   = image[y,x,2]            
            if red_channel <= 50 and green_channel >= 200 and blue_channel <= 50:
                image[y,x] = [0,0,0]    


def replace_black_with_color(image):          
    for y in range(w1):
        for x in range(h1):
            blue_channel  = image[y,x,0]
            green_channel = image[y,x,1]
            red_channel   = image[y,x,2]
            if np.all(image[y, x] == [0,0,0]):
                mean_color = mean_neighbour_pixel(image, x, y)
                image[y, x] = mean_color
      
    
def mean_neighbour_pixel(image, x , y):
    #Luam valoriile pixelilor vecini
    #neighbour_pixels = []
    neighbour_pixels = [
        image[y-1, x-1], #stanga-sus
        image[y-1, x],   #deasupra
        image[y-1, x+1], #dreapta-sus
    ]
            
    #Calculam culoarea media
    mean_color = np.mean(neighbour_pixels, axis=0).astype(int)
    
    return mean_color 

def images_To_Numpy(dataloader, max_iter):
    poze_test_data = []
    poze_preprocesate = []
    poze_masca_BGR = []
    for images, labels in dataloader:
        x_data = images.numpy() # tensor -> numpy
        poze_test_data.append(x_data)   #inseram toate pozele din dataloader intr-o lista
    poze_test_data = np.array(poze_test_data) #convertim lista in np.array
    #print(poze_test_data.shape)  #(992, 1, 3, 224, 224)    nr elem - batch_size - channels - height - width   

    
    contor = 0
    for i in range(poze_test_data.shape[0]):
        if contor < max_iter:
            img = poze_test_data[i,0]  #poza cu poza fara batch_size (3, 224, 224)
            img_v2 = poze_test_data[i] #poza cu poza CU batch pt partea a 2-a
            poze_preprocesate.append(img_v2)
            
            # Transpose the dimensions of img so that color channels are last
            img_T = np.transpose(img, (1,2,0))              # shape : (224, 224, 3)
        
            # Convert the image to the BGR color space
            img_BGR = cv2.cvtColor(img_T, cv2.COLOR_RGB2BGR)
            
                
            poze_masca_BGR.append(img_BGR) # bagam intr-o lista
        
        
            contor += 1
        
        
    cv2.destroyAllWindows()
    
    poze_masca_BGR = np.array(poze_masca_BGR) # facem numpy
    poze_preprocesate = np.array(poze_preprocesate)
    
    return poze_preprocesate

def afisare_openCV(images):
    cv2.imshow("Imagini", images)
    cv2.waitKey(1000)
    
w1 = 224
h1 = 224
#def segmentare_1 (dataloader, photo_number):
#    poze_preprocesate = images_To_Numpy(dataloader, photo_number)

def segmentare_1 (dataloader, poze_preprocesate):
    
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
    
    print(poze_preprocesate.shape)
    for i in range(poze_preprocesate.shape[0]):
        image = poze_preprocesate[i,0]
        image_T = np.transpose(image, (1,2,0))
        image_RGB = cv2.cvtColor(image_T, cv2.COLOR_BGR2RGB)
        image_RGB = (image_RGB - np.min(image_RGB)) / (np.max(image_RGB) - np.min(image_RGB))
        image_RGB = (image_RGB * 255).astype(np.uint8)
        
        #Detectam fete in imaginile color
        rects = detector(image_RGB, 1)
       # afisare_openCV(image_RGB)
        
        #Trecem prin fiecare detectare de fata
        for (j, rect) in enumerate(rects):
            #aplicam facial landmark detection => 68 (x,y) coordonate care mapeaza diverse trasaturi ale fetei
            shape = predictor(image_RGB, rect)
            shape = face_utils.shape_to_np(shape)   #shape_to_np(shape): Converts the facial landmark shape object (from dlib) to a NumPy array.
        
        
            #convertim dreptunghiul dlib => OpenCV bounding box [(x,y,w,h)]
            (x, y, w, h) = face_utils.rect_to_bb(rect) #rect_to_bb(rect): Converts a rectangle object (from dlib) to a tuple of bounding box coordinates (x, y, w, h).
        
        ##Vreau sa incerc sa fac o functie de taiere de gradul 2.
    
        #Pasul1. definim coodronatele de interes
        landmark_x1 = [shape[1,0], shape[27,0], shape[15,0]]
        landmark_y1 = [shape[1,1], shape[27,1], shape[15,1]]
    
        #Pasul2. Utilizăm curve_fit ca să potrivim funcția cu punctele de interes
        popt_1, _ = curve_fit(quadratic_function, landmark_x1, landmark_y1)
        
        #Pasul3. Generăm masca sintetică care va elimina masca din poză
        mask_shape = np.zeros_like(image_RGB, dtype = np.uint8)
        h1, w1 = image_RGB.shape[:2]

        
        for x1 in range(w1):
            for y1 in range(h1):
                if (
                    (y1 >= quadratic_function(x1, *popt_1)) and
                    (y1 <= shape[8,1]) and
                    (x1 >= shape[2,0] and x1 < shape[16,0])

                ):                                          # *popt sa luam element cu element din vector <=>
                    mask_shape[y1, x1, :] = 0               # <=> quadratic_function(x1,popt[0], popt[1], popt[2]) (în cazul acesta)
                else:
                    mask_shape[y1,x1,:] = 1



        # Aplicăm masca sintetică pe poza noastrab
        masked_image = image_RGB * mask_shape
        
        # Show the masked image
       # cv2.imshow("Masked Image", masked_image)
        #cv2.waitKey(2000)
        
    return masked_image
        
#def segmentare_2 (dataloader, photo_number):
    #poze_preprocesate_1 = images_To_Numpy(dataloader, photo_number)

def segmentare_2 (dataloader, poze_preprocesate_1):
    
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
    
    for i in range(poze_preprocesate_1.shape[0]):
        image = poze_preprocesate_1[i,0]
        image_T = np.transpose(image, (1,2,0))
        image_RGB = cv2.cvtColor(image_T, cv2.COLOR_BGR2RGB)
        image_RGB = (image_RGB - np.min(image_RGB)) / (np.max(image_RGB) - np.min(image_RGB))
        image_RGB = (image_RGB * 255).astype(np.uint8)
        
        #Detectam fete in imaginile color
        rects = detector(image_RGB, 1)
        #afisare_openCV(image_RGB)
        
        #Trecem prin fiecare detectare de fata
        for (j, rect) in enumerate(rects):
            #aplicam facial landmark detection => 68 (x,y) coordonate care mapeaza diverse trasaturi ale fetei
            shape = predictor(image_RGB, rect)
            shape = face_utils.shape_to_np(shape)   #shape_to_np(shape): Converts the facial landmark shape object (from dlib) to a NumPy array.
        
        
            #convertim dreptunghiul dlib => OpenCV bounding box [(x,y,w,h)]
            (x, y, w, h) = face_utils.rect_to_bb(rect) #rect_to_bb(rect): Converts a rectangle object (from dlib) to a tuple of bounding box coordinates (x, y, w, h).
        
        ##Vreau sa incerc sa fac o functie de taiere de gradul 2.
    
        #Pasul1. definim coodronatele de interes
        landmark_x1 = [shape[1,0], shape[27,0], shape[15,0]]
        landmark_y1 = [shape[1,1], shape[27,1], shape[15,1]]
    
        #Pasul2. Utilizăm curve_fit ca să potrivim funcția cu punctele de interes
        popt_1, _ = curve_fit(quadratic_function, landmark_x1, landmark_y1)
        
        #Pasul3. Generăm masca sintetică care va elimina masca din poză
        mask_shape_1 = np.zeros_like(image_RGB, dtype = np.uint8)
        h1, w1 = image_RGB.shape[:2]

        
        for x1 in range(w1):
            for y1 in range(h1):
                if (
                    (y1 >= quadratic_function(x1, *popt_1)) and
                    (y1 <= shape[8,1]) and
                    (x1 >= shape[2,0] and x1 < shape[16,0])

                ):                                          # *popt sa luam element cu element din vector <=>
                    mask_shape_1[y1, x1, :] = 1               # <=> quadratic_function(x1,popt[0], popt[1], popt[2]) (în cazul acesta)
                else:
                    mask_shape_1[y1,x1,:] = 0



        # Aplicăm masca sintetică pe poza noastrab
        masked_image_1 = image_RGB * mask_shape_1
        
         # Show the masked image
        #cv2.imshow("Masked Image", masked_image_1)
        #cv2.waitKey(2000)
        
    return masked_image_1   

def afisare_finala_openCV(images):
    cv2.imshow("Imagini", images)
    cv2.waitKey(10000)
    
def quadratic_function(x, a, b, c):
    return a * x**2 + b * x + c


#1. seturi de date 
transform_test, image_Datasets_WithMask,image_Datasets_WithoutMask,dataloaders_WithMask,dataloaders_WithoutMask = import_and_do_datasets(URL1, URL2)

#2 model_masca
prediction_mask = model_mask_output("E:\\Profil\\Desktop\\Licenta\\ISIA\\LICENTA\\persoane\\Dataset\\Test\\Buli\\1.png", 'buli_nr_1.pth')
print(prediction_mask)

if prediction_mask == 0:
    print("Persoana nu poartă mască")
else:
    #3 model_persoana
    prediction_people = model_people("E:\\Profil\\Desktop\\Licenta\\ISIA\\LICENTA\\persoane\\Dataset\\Test\\Buli\\1.png", 'model_persoane.pth')
    person = check_person(prediction_people)
    print(person)
    
    #4 segmentam persoana
    poze_preprocesate = images_To_Numpy(dataloaders_WithMask, person)
    poze_preprocesate_1 = images_To_Numpy(dataloaders_WithoutMask, person)

    cu_masca = segmentare_1(dataloaders_WithMask, poze_preprocesate)
    fara_masca = segmentare_2(dataloaders_WithoutMask, poze_preprocesate_1)
    
    #5
    reconstructie = cu_masca + fara_masca
    replace_red_with_color(reconstructie)
    replace_cyan_with_color(reconstructie)
    replace_green_with_color(reconstructie)
    replace_black_with_color(reconstructie)
    afisare_finala_openCV(reconstructie)
    
    cv2.destroyAllWindows() 

1
2
(2, 1, 3, 224, 224)
